# LAB 4

### Importação das bibliotecas para gravação e conexão com o Blob Storage

In [118]:
from azure.identity import DefaultAzureCredential
from azure.storage.blob import BlobServiceClient, BlobClient
import fastparquet
import pandas as pd

### Conta e container necessários para acesso ao Blob

In [119]:
conta_armazenamento = 'XXXXXXXXX'
container = 'XXXXXXXXXX'

### Criando uma credencial

In [120]:
token_credential = DefaultAzureCredential()
print(token_credential)

### Conexão com o Blob 

In [121]:
blob_service_client = BlobServiceClient(account_url=f"https://{conta_armazenamento}.blob.core.windows.net/{container}", credential=token_credential)
print(blob_service_client)

# Chave padrão de acesso ao Blob Storage
chave = "XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"

### Eventos

#### Coleta de dados

In [122]:
df_eventos = pd.read_json('http://museus.cultura.gov.br/api/event/find?@select=*')
df_eventos.head()

,id,name,shortDescription,longDescription,rules,createTimestamp,status,updateTimestamp,owner,project,...,twitter,instagram,linkedin,spotify,youtube,pinterest,event_attendance,opportunityTabName,useOpportunityTab,_relatedOpportunities
0,448,São José de todos os esportes,"Estrategicamente montada no corredor central, ...","Oficialmente, a história do esporte começou a ...",NaN,"{'date': '2017-02-13 15:25:28.000000', 'timezo...",1,"{'date': '2017-05-25 23:24:10.000000', 'timezo...",13119.0,NaN,...,None,None,NaN,NaN,None,NaN,None,None,None,NaN
1,473,FESTA DA POESIA,Este evento surge como promotor de processos i...,"Neste ano proporemos, através deste, as interv...",NaN,"{'date': '2017-03-02 15:04:46.000000', 'timezo...",1,"{'date': '2017-03-02 15:36:48.000000', 'timezo...",21987.0,NaN,...,None,None,NaN,NaN,None,NaN,None,None,None,NaN
2,988,Sementes de Poesia,"Neste domingo, 16 de julho de 2017 de 10 às 12...",Sementes de Poesia em homenagem ao Samba e Tan...,NaN,"{'date': '2017-07-12 18:11:13.000000', 'timezo...",1,"{'date': '2017-07-12 18:22:26.000000', 'timezo...",20193.0,NaN,...,None,None,NaN,NaN,None,NaN,None,None,None,NaN
3,1044,OFICINA DE FÉRIAS NO MUSEU DA FMRB,Oficinas de Férias,FUNDAÇÃO DA MEMÓRIA REPUBLICANA BRASILEIRA\n\n...,NaN,"{'date': '2017-07-20 11:09:08.000000', 'timezo...",1,"{'date': '2017-07-20 11:09:08.000000', 'timezo...",NaN,NaN,...,None,None,NaN,NaN,None,NaN,None,None,None,NaN
4,1066,Limpando os Manguezais 2017,"Limpeza, triagem e retirada do lixo, na foz do...",Ontem tivemos o orgulho de promover o Limpando...,NaN,"{'date': '2017-07-29 16:57:32.000000', 'timezo...",1,"{'date': '2017-07-29 17:04:44.000000', 'timezo...",NaN,NaN,...,None,None,NaN,NaN,None,NaN,None,None,None,NaN


In [123]:
# Listando todas as colunas do df_eventos
colunas = [col for col in df_eventos.columns]
print(colunas)

['id', 'name', 'shortDescription', 'longDescription', 'rules', 'createTimestamp', 'status', 'updateTimestamp', 'owner', 'project', 'subsite', 'subTitle', 'classificacaoEtaria', 'telefonePublico', 'traducaoLibras', 'descricaoSonora', 'num_sniic', 'occurrences', 'relatedOpportunities', 'terms', 'type', 'subsiteId', 'registrationInfo', 'preco', 'site', 'facebook', 'twitter', 'instagram', 'linkedin', 'spotify', 'youtube', 'pinterest', 'event_attendance', 'opportunityTabName', 'useOpportunityTab', '_relatedOpportunities']


In [124]:
# Verificando os valores nulos e não nulos
df_eventos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1556 entries, 0 to 1555
Data columns (total 36 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     1556 non-null   int64  
 1   name                   1556 non-null   object 
 2   shortDescription       1556 non-null   object 
 3   longDescription        1555 non-null   object 
 4   rules                  0 non-null      float64
 5   createTimestamp        1556 non-null   object 
 6   status                 1556 non-null   int64  
 7   updateTimestamp        1524 non-null   object 
 8   owner                  1536 non-null   float64
 9   project                129 non-null    float64
 10  subsite                1556 non-null   int64  
 11  subTitle               824 non-null    object 
 12  classificacaoEtaria    1556 non-null   object 
 13  telefonePublico        1021 non-null   object 
 14  traducaoLibras         469 non-null    object 
 15  desc

#### Limpeza de dados

In [125]:
# Removendo colunas vazias, com poucos dados e colunas que não serão utilizadas
df_eventos.drop(
columns=['createTimestamp', 'status', 'updateTimestamp', 'owner', 'subsite', 'num_sniic', 'occurrences',
 'relatedOpportunities','terms','type','rules', 'project', 'subsiteId', 'preco', 'facebook', 'twitter', 'instagram',
 'linkedin', 'spotify', 'youtube', 'pinterest', 'event_attendance','opportunityTabName','useOpportunityTab',
 '_relatedOpportunities'],
 inplace=True)

In [126]:
df_eventos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1556 entries, 0 to 1555
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   id                   1556 non-null   int64 
 1   name                 1556 non-null   object
 2   shortDescription     1556 non-null   object
 3   longDescription      1555 non-null   object
 4   subTitle             824 non-null    object
 5   classificacaoEtaria  1556 non-null   object
 6   telefonePublico      1021 non-null   object
 7   traducaoLibras       469 non-null    object
 8   descricaoSonora      454 non-null    object
 9   registrationInfo     523 non-null    object
 10  site                 470 non-null    object
dtypes: int64(1), object(10)
memory usage: 133.8+ KB


In [127]:
df_eventos['id'].duplicated().sum()


0

In [128]:
# Preenchendo os valores vazios de todas as colunas por "Não informado"
df_eventos.fillna('Não informado', inplace=True)


##### Coluna 'shortDescription'

In [129]:
#shortDescription = df_eventos['shortDescription'].unique().tolist()
df_eventos['shortDescription'] = df_eventos['shortDescription'].replace('\n\n\n', ' ',regex=True).replace('\n\n', ' ',regex=True)\
        .replace('\n', ' ',regex=True).replace('.', 'Não informado').replace('ghjj', 'Não informado').replace('ojojo9jojojoojoojjojojojo', 'Não informado')\
        .replace('teste', 'Não informado').replace('Teste', 'Não informado').replace('dfhgfjfjjj', 'Não informado').replace('fvfdvd', 'Não informado')\
        .replace('LL', 'Não informado').replace('-', 'Não informado').replace('   ', 'Não informado',regex=True).replace('  ', 'Não informado',regex=True)

# vistos 
 # ..., +, =, $, ...

# Limpar 
 # \n\n\n, n\n, \n, '.', 'ghjj', 'ojojo9jojojoojoojjojojojo', 'teste', 'dfhgfjfjjj', 'Teste', 'fvfdvd', 'LL', '-', '   ', '  '

##### Coluna 'longDescription'

In [130]:
df_eventos['longDescription'] = df_eventos['longDescription'].str.strip()

df_eventos['longDescription'] = df_eventos['longDescription'].replace('\n\n\n', ' ',regex=True).replace('\n\n', ' ',regex=True)\
        .replace('\n', ' ',regex=True).replace('     ', ' ',regex=True).replace('   ', ' ',regex=True).replace('  ', ' ',regex=True)\
        .replace('~', '',regex=True).replace('', 'Não informado')

# '', 
# regex --> \n, \n\n, \n\n\n, \n\n\n\n, \n\n\n\n\n\n\n, \n~~~~\n, None, ~

##### Coluna 'subTitle'

In [131]:
df_eventos['subTitle'] = df_eventos['subTitle'].replace('teste123','Não informado').replace('','Não informado').replace('  ', ' ', regex=True)

#'', 'teste123', '  '
# regex -> 

##### Coluna 'telefonePublico'

In [132]:
df_eventos['telefonePublico'] = df_eventos['telefonePublico'].replace('','Não informado').replace('(00) 0000-0000','Não informado')

# '', '(00) 0000-0000',
#

##### Coluna 'descricaoSonora'

In [133]:
#desc_s = df_eventos['descricaoSonora'].unique().tolist() 
#desc_s

df_eventos['descricaoSonora'] = df_eventos['descricaoSonora'].replace('','Não informado')

##### Coluna 'registrationInfo'

In [134]:
gratuito = ['Gratuita', 'gratuita', 'Entrada franca', 'sem inscrições', 'Gratuitas', 'Livre', 'Inscrições Gratuitas', 'entrada grátis', 'entradas grátisi',\
     'A entrada é grátis', 'entradas grátis', 'Entradas francas', 'Inscrições gratuitas', 'Livre acesso', 'Retirar ingressos gratuitos.','grátis', 'Grátis',\
     'entradas gratuitas', 'A entrada é gratuita', 'Entrada Franca', 'Sem inscrições', 'gratuitas', 'Entrada franca.','Inscrições gratuitas!','Gratuitas!',\
     'Entradas Grátis', 'Entrada gratuita', 'Entradas Gratuitas', 'Entradas gratuitas', 'As entradas são gratuitas','Não precisa inscrições e a entrada é grátis.',
     'Entrada grátis e não precisa se inscrever.', 'Entrada gratuita. Não precisa se inscrever.', 'entradas gratuitas e não precisa se inscrever', 'Não precisa se inscrever. A entrada é grátis',\
     'Não precisa se inscrever. entrada franca', 'Não precisa se inscrever. entrada grátis', 'Nâo precisa se inscrever. Entrada Framca',  'não precisa se inscrever.',
     'Não precisa se inscrever. Entrada gratuita.', 'NÃO PRECISA SE INSCREVER', 'Inscricoes no site', 'não precisa inscrições', 'não precisa se inscrever', 'Não precisa inscrever-se',\
     'Não precisa se inscrever. Aberto ao público', 'Não precisa se inscrever',  'Sem inscrição . Entrada franca.', 'Sem inscrições. Entrada gratuita', 'Sem inscrições . Entrada Franca.',\
     'Sem inscrições. Entrada franca', 'Sem inscrições. Entrada franca.', 'Sem inscrição. Entrada Franca', 'Sem Inscrição. Entrada franca.', 'Não precisa se inscrever.',]

for g in gratuito:
    df_eventos['registrationInfo'] = df_eventos['registrationInfo'].replace({g},'Gratuito')


df_eventos['registrationInfo'] = df_eventos['registrationInfo'].replace('Doing as entradas são grátis', 'Domingo a entrada é gratuita')\
    .replace('','Não informado').replace('---','Não informado').replace('não se aplica','Não informado').replace('Não se aplica','Não informado')\
    .replace('Nada declarar','Não informado').replace('\n',' ',regex=True)


##### Coluna 'site'

In [135]:
df_eventos['site'] = df_eventos['site'].replace('','Não informado')


##### Renomear colunas

In [136]:
lista = [i for i in df_eventos.columns]
print(lista)

['id', 'name', 'shortDescription', 'longDescription', 'subTitle', 'classificacaoEtaria', 'telefonePublico', 'traducaoLibras', 'descricaoSonora', 'registrationInfo', 'site']


In [137]:
df_eventos.rename(columns={'id':'Id', 'name':'Nome_evento','shortDescription':'Curta_desc', 'longDescription':'Longa_desc', 'subTitle':'Subtitulo',
                  'classificacaoEtaria':'Class_etaria', 'telefonePublico':'Telefone_pub', 'traducaoLibras':'Trad_libras',
                  'descricaoSonora':'Desc_sonora', 'registrationInfo': 'Obs', 'site':'Sites'}, inplace=True)

In [138]:
df_eventos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1556 entries, 0 to 1555
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Id            1556 non-null   int64 
 1   Nome_evento   1556 non-null   object
 2   Curta_desc    1556 non-null   object
 3   Longa_desc    1556 non-null   object
 4   Subtitulo     1556 non-null   object
 5   Class_etaria  1556 non-null   object
 6   Telefone_pub  1556 non-null   object
 7   Trad_libras   1556 non-null   object
 8   Desc_sonora   1556 non-null   object
 9   Obs           1556 non-null   object
 10  Sites         1556 non-null   object
dtypes: int64(1), object(10)
memory usage: 133.8+ KB


In [139]:
df_eventos.head()

,Id,Nome_evento,Curta_desc,Longa_desc,Subtitulo,Class_etaria,Telefone_pub,Trad_libras,Desc_sonora,Obs,Sites
0,448,São José de todos os esportes,"Estrategicamente montada no corredor central, ...","Oficialmente, a história do esporte começou a ...",Exposição de longa duração,Livre,(12) 3921-4112,Não,Não,Não informado,Não informado
1,473,FESTA DA POESIA,Este evento surge como promotor de processos i...,"Neste ano proporemos, através deste, as interv...",Não informado,10 anos,(84) 99667-0060,Não informado,Não informado,Não informado,Não informado
2,988,Sementes de Poesia,"Neste domingo, 16 de julho de 2017 de 10 às 12...",Sementes de Poesia em homenagem ao Samba e Tan...,Edição 108 - Entre o Samba e o Tango,Livre,(31) 98838-7367,Não informado,Não informado,Não informado,http://www.museunacionaldapoesia.blogspot.com
3,1044,OFICINA DE FÉRIAS NO MUSEU DA FMRB,Oficinas de Férias,FUNDAÇÃO DA MEMÓRIA REPUBLICANA BRASILEIRA Pro...,FÉRIAS NO MUSEU,14 anos,(98) 3221-3724,Não informado,Não informado,Inscrições estão abertas. Faça sua inscrição ...,http://www.fmrb.ma.gov.br
4,1066,Limpando os Manguezais 2017,"Limpeza, triagem e retirada do lixo, na foz do...",Ontem tivemos o orgulho de promover o Limpando...,"Mangue Limpo, Mangue vivo",Livre,Não informado,Não informado,Não informado,Não informado,https://www.facebook.com/ecomuseunaturaldomang...


### Ocorrências

In [140]:
ocorrencias = pd.read_json('http://museus.cultura.gov.br/api/event/find?@select=occurrences.*')
ocorrencias.head(10)

,id,occurrences
0,448,[]
1,473,[]
2,988,[]
3,1044,[]
4,1066,[]
5,1067,[]
6,1068,[]
7,1309,[]
8,1325,"[{'id': 739, 'frequency': 'daily', 'separation..."
9,1327,"[{'id': 742, 'frequency': 'weekly', 'separatio..."


#### Limpeza de dados

Preenchendo as listas vazias da coluna 'occurrences'

In [141]:
preencher = [{'id': None, 'frequency': None, 'separation': None, 'count': None, 'timezoneName': None, 'eventId': None, 'spaceId': None, 'rule': {'spaceId': None, 'startsAt': None, 'duration': None, 'endsAt': None, 'frequency': None, 'startsOn': None, 'until': None, 'description': None, 'price': None}, 'status': None, 'space': None, 'terms': None, 'startsOn': None, 'endsOn': None, 'startsAt': None, 'endsAt': None, 'until': None}]

ocorrencias.occurrences = [oc if oc != [] else preencher  for oc in ocorrencias.occurrences]

In [142]:
var = ocorrencias['occurrences'].loc[0]
var

[{'id': None,
  'frequency': None,
  'separation': None,
  'count': None,
  'timezoneName': None,
  'eventId': None,
  'spaceId': None,
  'rule': {'spaceId': None,
   'startsAt': None,
   'duration': None,
   'endsAt': None,
   'frequency': None,
   'startsOn': None,
   'until': None,
   'description': None,
   'price': None},
  'status': None,
  'space': None,
  'terms': None,
  'startsOn': None,
  'endsOn': None,
  'startsAt': None,
  'endsAt': None,
  'until': None}]

In [143]:
ocorrencias.occurrences[8][0]['spaceId']

202940

In [144]:
teste = ocorrencias.occurrences[300]
print(teste)


[{'id': 2178, 'frequency': 'once', 'separation': 1, 'count': None, 'timezoneName': 'Etc/UTC', 'eventId': 2798, 'spaceId': 201228, 'rule': {'spaceId': '201228', 'startsAt': '14:00', 'duration': 60, 'endsAt': '15:00', 'frequency': 'once', 'startsOn': '2018-05-20', 'until': '', 'description': 'Dia 20 de maio de 2018 das 14:00 às 15:00 h', 'price': 'grátis'}, 'status': 1, 'space': 201228, 'terms': None, 'startsOn': None, 'endsOn': None, 'startsAt': None, 'endsAt': None, 'until': None}]


Explode na coluna 'occurrences'

In [145]:
ocorrencias['Frequencia'] = [frequencia[0]['frequency'] for frequencia in (ocorrencias.occurrences)]
ocorrencias['Id_museu'] = [id_museu[0]['spaceId'] for id_museu in (ocorrencias.occurrences)]
ocorrencias['Hora_inicio'] = [hora_inicio[0]['rule']['startsAt'] for hora_inicio in (ocorrencias.occurrences)]
ocorrencias['Hora_fim'] = [hora_fim[0]['rule']['endsAt'] for hora_fim in (ocorrencias.occurrences)]
ocorrencias['Inicio'] = [inicio[0]['rule']['startsOn'] for inicio in (ocorrencias.occurrences)]
ocorrencias['Fim'] = [fim[0]['rule']['until'] for fim in (ocorrencias.occurrences)]
ocorrencias['Descript'] = [desc[0]['rule']['description'] for desc in (ocorrencias.occurrences)]
ocorrencias['Preco'] = [preco[0]['rule']['price'] for preco in (ocorrencias.occurrences)]


#ocorrencias['id'] == Id_eventos

In [146]:
ocorrencias[5:14]

,id,occurrences,Frequencia,Id_museu,Hora_inicio,Hora_fim,Inicio,Fim,Descript,Preco
5,1067,"[{'id': None, 'frequency': None, 'separation':...",None,NaN,None,None,None,None,None,None
6,1068,"[{'id': None, 'frequency': None, 'separation':...",None,NaN,None,None,None,None,None,None
7,1309,"[{'id': None, 'frequency': None, 'separation':...",None,NaN,None,None,None,None,None,None
8,1325,"[{'id': 739, 'frequency': 'daily', 'separation...",daily,202940.0,10:00,18:00,2017-09-06,2017-10-22,Diariamente de 6 de setembro a 22 de outubro d...,De terça a sexta: R$ 10 e R$ 5 (meia-entrada)....
9,1327,"[{'id': 742, 'frequency': 'weekly', 'separatio...",weekly,202940.0,11:00,13:00,2017-09-05,2017-09-19,"5 e 19 de setembro, terças-feiras às 11h",Gratuito
10,1328,"[{'id': 743, 'frequency': 'weekly', 'separatio...",weekly,202940.0,10:30,12:00,2017-09-06,2017-09-20,"6 e 20 de setembro, quartas-feiras às 10h30",Gratuito
11,1329,"[{'id': 744, 'frequency': 'weekly', 'separatio...",weekly,202940.0,10:30,12:00,2017-09-13,2017-09-27,"13 e 27 de setembro, quartas-feiras às 10h30",Gratuito
12,1330,"[{'id': 745, 'frequency': 'once', 'separation'...",once,202940.0,13:30,15:00,2017-09-14,,Dia 14 de setembro de 2017 às 13:30,Gratuito
13,1331,"[{'id': 746, 'frequency': 'weekly', 'separatio...",weekly,202940.0,10:30,12:00,2017-09-21,2017-09-28,Toda quinta-feira de 21 a 28 de setembro de 20...,Gratuito


In [147]:
len(ocorrencias.loc[ocorrencias.Frequencia == 'daily'])

302

Dropando as linhas com valores nulos e dropando a coluna 'occurrences'

In [148]:
#ocorrencias.fillna('Não informado', inplace=True)
ocorrencias.dropna(inplace=True)
ocorrencias.drop(columns=['occurrences'], inplace = True)


In [149]:
ocorrencias.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1386 entries, 8 to 1555
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           1386 non-null   int64  
 1   Frequencia   1386 non-null   object 
 2   Id_museu     1386 non-null   float64
 3   Hora_inicio  1386 non-null   object 
 4   Hora_fim     1386 non-null   object 
 5   Inicio       1386 non-null   object 
 6   Fim          1386 non-null   object 
 7   Descript     1386 non-null   object 
 8   Preco        1386 non-null   object 
dtypes: float64(1), int64(1), object(7)
memory usage: 108.3+ KB


##### Coluna 'id'

Essa coluna possui os id's do evento

In [150]:
ocorrencias.rename(columns={'id':'Id_evento'}, inplace = True)

##### Coluna 'Fim'

Coluna 'Fim' possui valores vazios onde a Coluna 'Frequencia' é igual a 'once'

In [151]:
ocorrencias.loc[ocorrencias.Fim == '']

,Id_evento,Frequencia,Id_museu,Hora_inicio,Hora_fim,Inicio,Fim,Descript,Preco
12,1330,once,202940.0,13:30,15:00,2017-09-14,,Dia 14 de setembro de 2017 às 13:30,Gratuito
14,1333,once,202940.0,14:30,16:00,2017-09-10,,Dia 10 de setembro de 2017 às 14:30,Gratuito
16,1345,once,202940.0,14:30,16:30,2017-09-30,,Dia 30 de setembro de 2017 às 14:30,Gratuito
18,1348,once,202940.0,11:00,12:30,2017-09-10,,Dia 10 de setembro de 2017 às 11:00,Gratuito
19,1349,once,202940.0,11:00,12:30,2017-09-17,,Dia 17 de setembro de 2017 às 11:00,Gratuito
...,...,...,...,...,...,...,...,...,...
1551,8216,once,6091.0,17:30,20:00,2022-09-20,,"Dias 20 e 21 de setembro de 2022, às 17:30",Gratuito
1552,8217,once,6091.0,14:00,15:00,2022-11-16,,Dia 16 de novembro de 2022 às 14:00,Gratuito
1553,8218,once,6091.0,14:00,15:00,2022-11-18,,Dia 18 de novembro de 2022 às 14:00,Gratuito
1554,8219,once,6091.0,10:00,11:00,2022-09-20,,Dia 20 de setembro de 2022 às 10:00,Gratuito


In [152]:
# Conferindo a quantidade 
print(f"Coluna 'Fim'=='' --> {len(ocorrencias.loc[ocorrencias.Fim == ''])}") # 915
print(f"Coluna 'Frequencia'=='once' --> {len(ocorrencias.loc[ocorrencias.Frequencia == 'once'])}") # 915


Coluna 'Fim'=='' --> 915
Coluna 'Frequencia'=='once' --> 915


Tratamento da Coluna 'Fim', onde a Coluna 'Frequencia' é igual a 'once'

In [153]:
# Preenchimento da coluna 'Fim' onde o valor é vazio, pela data da coluna 'Inicio'
ocorrencias.Fim = [i  if freq == 'once'  else f  for f, freq, i in zip(ocorrencias['Fim'], ocorrencias['Frequencia'], ocorrencias['Inicio'])] 


In [154]:
display(ocorrencias[10:20].loc[ocorrencias.Frequencia=='once'])

,Id_evento,Frequencia,Id_museu,Hora_inicio,Hora_fim,Inicio,Fim,Descript,Preco
18,1348,once,202940.0,11:00,12:30,2017-09-10,2017-09-10,Dia 10 de setembro de 2017 às 11:00,Gratuito
19,1349,once,202940.0,11:00,12:30,2017-09-17,2017-09-17,Dia 17 de setembro de 2017 às 11:00,Gratuito
20,1350,once,202940.0,11:00,12:30,2017-09-24,2017-09-24,Dia 24 de setembro de 2017 às 11:00,Gratuito


##### Coluna 'Preco'

Tratamento de valores incorretos, despadronizados, não informados

In [155]:
ocorrencias['Preco'] = ocorrencias['Preco'].apply(lambda x: x.capitalize()).str.strip()

ocorrencias['Preco'] = ocorrencias['Preco'].replace('','Não informado').replace(' ','Não informado').replace('1144768,83','Não informado')\
                            .replace('r\$','R$', regex=True).replace('-','Não informado')

Gratuito = ['0','0000', '00', '0,00', 'R$ 00,00', 'R$ 0,00', '00,00', 'R$ 0,00','Entrada gratuita.','Entrada gratuita',\
            'Entrada franca', 'Grátis','Gratuita', 'Entrada franca.']
for g in Gratuito:
    ocorrencias['Preco'] = ocorrencias['Preco'].replace({g},'Gratuito')



# 'Não informado' -> '', ' ', '-', 1144768,83
# regex -->  'r$'
# Gratuito -->  '0','0000', '00', '0,00', 'R$ 00,00',  'R$ 0,00',  '00,00',  'R$ 0,00',


##### Coluna 'Frequencia'

Traduzindo os valores da coluna 'Frequencia'

In [156]:
ocorrencias['Frequencia'] = ocorrencias['Frequencia'].replace('daily','Diário').replace('weekly','Semanal').replace('once','Uma vez')


##### Colunas 'Inicio' e 'Fim'

Conversão das colunas para o formato de data

In [157]:
ocorrencias['Inicio'] = pd.to_datetime(ocorrencias['Inicio'])
ocorrencias['Fim'] = pd.to_datetime(ocorrencias['Fim'])

In [158]:
ocorrencias.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1386 entries, 8 to 1555
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Id_evento    1386 non-null   int64         
 1   Frequencia   1386 non-null   object        
 2   Id_museu     1386 non-null   float64       
 3   Hora_inicio  1386 non-null   object        
 4   Hora_fim     1386 non-null   object        
 5   Inicio       1386 non-null   datetime64[ns]
 6   Fim          1386 non-null   datetime64[ns]
 7   Descript     1386 non-null   object        
 8   Preco        1386 non-null   object        
dtypes: datetime64[ns](2), float64(1), int64(1), object(5)
memory usage: 140.6+ KB


In [159]:
ocorrencias.head(6)

,Id_evento,Frequencia,Id_museu,Hora_inicio,Hora_fim,Inicio,Fim,Descript,Preco
8,1325,Diário,202940.0,10:00,18:00,2017-09-06,2017-10-22,Diariamente de 6 de setembro a 22 de outubro d...,De terça a sexta: R$ 10 e R$ 5 (meia-entrada)....
9,1327,Semanal,202940.0,11:00,13:00,2017-09-05,2017-09-19,"5 e 19 de setembro, terças-feiras às 11h",Gratuito
10,1328,Semanal,202940.0,10:30,12:00,2017-09-06,2017-09-20,"6 e 20 de setembro, quartas-feiras às 10h30",Gratuito
11,1329,Semanal,202940.0,10:30,12:00,2017-09-13,2017-09-27,"13 e 27 de setembro, quartas-feiras às 10h30",Gratuito
12,1330,Uma vez,202940.0,13:30,15:00,2017-09-14,2017-09-14,Dia 14 de setembro de 2017 às 13:30,Gratuito
13,1331,Semanal,202940.0,10:30,12:00,2017-09-21,2017-09-28,Toda quinta-feira de 21 a 28 de setembro de 20...,Gratuito


In [160]:
df_eventos.head(6)

,Id,Nome_evento,Curta_desc,Longa_desc,Subtitulo,Class_etaria,Telefone_pub,Trad_libras,Desc_sonora,Obs,Sites
0,448,São José de todos os esportes,"Estrategicamente montada no corredor central, ...","Oficialmente, a história do esporte começou a ...",Exposição de longa duração,Livre,(12) 3921-4112,Não,Não,Não informado,Não informado
1,473,FESTA DA POESIA,Este evento surge como promotor de processos i...,"Neste ano proporemos, através deste, as interv...",Não informado,10 anos,(84) 99667-0060,Não informado,Não informado,Não informado,Não informado
2,988,Sementes de Poesia,"Neste domingo, 16 de julho de 2017 de 10 às 12...",Sementes de Poesia em homenagem ao Samba e Tan...,Edição 108 - Entre o Samba e o Tango,Livre,(31) 98838-7367,Não informado,Não informado,Não informado,http://www.museunacionaldapoesia.blogspot.com
3,1044,OFICINA DE FÉRIAS NO MUSEU DA FMRB,Oficinas de Férias,FUNDAÇÃO DA MEMÓRIA REPUBLICANA BRASILEIRA Pro...,FÉRIAS NO MUSEU,14 anos,(98) 3221-3724,Não informado,Não informado,Inscrições estão abertas. Faça sua inscrição ...,http://www.fmrb.ma.gov.br
4,1066,Limpando os Manguezais 2017,"Limpeza, triagem e retirada do lixo, na foz do...",Ontem tivemos o orgulho de promover o Limpando...,"Mangue Limpo, Mangue vivo",Livre,Não informado,Não informado,Não informado,Não informado,https://www.facebook.com/ecomuseunaturaldomang...
5,1067,9 Semana de Museus - 2011,Exposição com o Museu Móvel - levando cultura ...,Não informado,11 Semana de Museus,Livre,Não informado,Não informado,Não informado,Não informado,Não informado


### Juntando os dois Dataframes

In [161]:
eventos = df_eventos.merge(ocorrencias, left_on='Id', right_on='Id_evento', how='outer')
eventos[6:12]

,Id,Nome_evento,Curta_desc,Longa_desc,Subtitulo,Class_etaria,Telefone_pub,Trad_libras,Desc_sonora,Obs,Sites,Id_evento,Frequencia,Id_museu,Hora_inicio,Hora_fim,Inicio,Fim,Descript,Preco
6,1068,10 Semana de Museus,Visitação ao Ecomuseu e aula de campo,"Aulas de campo, exposição e exibição de filmes...",Não informado,Livre,Não informado,Não informado,Não informado,Não informado,Não informado,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN
7,1309,5ª Exposição Itinerante do Museu do RPG,23/09/2017 às 10:00 Endereço: R. Gonçalves Dia...,Venha conhecer nosso acervo físico: Livros de ...,"R. Gonçalves Dias, 1922 - sala 301 - Lourdes, ...",Livre,(31) 3072-7401,Não,Não,"Gratuitas no evento - R. Gonçalves Dias, 1922 ...",https://www.facebook.com/events/1738599869773715,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN
8,1325,Mostra | Diálogo Design: Polônia Brasil,"Com a curadoria de Magda Kochanowska, Gabriel ...","Dois países, duas culturas e duas histórias se...",Não informado,Livre,(11) 3032-3727,Não informado,Não informado,Não informado,http://www.mcb.org.br,1325.0,Diário,202940.0,10:00,18:00,2017-09-06,2017-10-22,Diariamente de 6 de setembro a 22 de outubro d...,De terça a sexta: R$ 10 e R$ 5 (meia-entrada)....
9,1327,Educativo MCB | A casa estampada,"A artista Monica Schoenacker oferecerá, ao púb...","A artista Monica Schoenacker oferecerá, ao púb...",Não informado,Livre,(11) 3032-3727,Não informado,Não informado,Não informado,http://www.mcb.org.br,1327.0,Semanal,202940.0,11:00,13:00,2017-09-05,2017-09-19,"5 e 19 de setembro, terças-feiras às 11h",Gratuito
10,1328,Educativo MCB | Desenho sua casa,"Marcos Gorgatti, artista educador do MCB, faz ...","Marcos Gorgatti, artista educador do MCB, faz ...",Não informado,Livre,(11) 3032-3727,Não informado,Não informado,Não informado,http://www.mcb.org.br,1328.0,Semanal,202940.0,10:30,12:00,2017-09-06,2017-09-20,"6 e 20 de setembro, quartas-feiras às 10h30",Gratuito
11,1329,Educativo MCB | As palavras e as casas,A partir de relatos afetivos relacionados ao m...,A partir de relatos afetivos relacionados ao m...,Não informado,Livre,(11) 3032-3727,Não informado,Não informado,Não informado,http://www.mcb.org.br,1329.0,Semanal,202940.0,10:30,12:00,2017-09-13,2017-09-27,"13 e 27 de setembro, quartas-feiras às 10h30",Gratuito


Dropando os valores nulos

In [162]:
eventos.dropna(inplace=True)

Resetando o Index

In [163]:
eventos.reset_index(drop=True, inplace=True)
eventos.head(5)

,Id,Nome_evento,Curta_desc,Longa_desc,Subtitulo,Class_etaria,Telefone_pub,Trad_libras,Desc_sonora,Obs,Sites,Id_evento,Frequencia,Id_museu,Hora_inicio,Hora_fim,Inicio,Fim,Descript,Preco
0,1325,Mostra | Diálogo Design: Polônia Brasil,"Com a curadoria de Magda Kochanowska, Gabriel ...","Dois países, duas culturas e duas histórias se...",Não informado,Livre,(11) 3032-3727,Não informado,Não informado,Não informado,http://www.mcb.org.br,1325.0,Diário,202940.0,10:00,18:00,2017-09-06,2017-10-22,Diariamente de 6 de setembro a 22 de outubro d...,De terça a sexta: R$ 10 e R$ 5 (meia-entrada)....
1,1327,Educativo MCB | A casa estampada,"A artista Monica Schoenacker oferecerá, ao púb...","A artista Monica Schoenacker oferecerá, ao púb...",Não informado,Livre,(11) 3032-3727,Não informado,Não informado,Não informado,http://www.mcb.org.br,1327.0,Semanal,202940.0,11:00,13:00,2017-09-05,2017-09-19,"5 e 19 de setembro, terças-feiras às 11h",Gratuito
2,1328,Educativo MCB | Desenho sua casa,"Marcos Gorgatti, artista educador do MCB, faz ...","Marcos Gorgatti, artista educador do MCB, faz ...",Não informado,Livre,(11) 3032-3727,Não informado,Não informado,Não informado,http://www.mcb.org.br,1328.0,Semanal,202940.0,10:30,12:00,2017-09-06,2017-09-20,"6 e 20 de setembro, quartas-feiras às 10h30",Gratuito
3,1329,Educativo MCB | As palavras e as casas,A partir de relatos afetivos relacionados ao m...,A partir de relatos afetivos relacionados ao m...,Não informado,Livre,(11) 3032-3727,Não informado,Não informado,Não informado,http://www.mcb.org.br,1329.0,Semanal,202940.0,10:30,12:00,2017-09-13,2017-09-27,"13 e 27 de setembro, quartas-feiras às 10h30",Gratuito
4,1330,Educativo MCB | Casa Bordada,"A partir de imagens de peças do acervo do MCB,...","A partir de imagens de peças do acervo do MCB,...",Não informado,Livre,(11) 3032-3727,Não informado,Não informado,Não informado,http://www.mcb.org.br,1330.0,Uma vez,202940.0,13:30,15:00,2017-09-14,2017-09-14,Dia 14 de setembro de 2017 às 13:30,Gratuito


In [164]:
eventos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1386 entries, 0 to 1385
Data columns (total 20 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Id            1386 non-null   int64         
 1   Nome_evento   1386 non-null   object        
 2   Curta_desc    1386 non-null   object        
 3   Longa_desc    1386 non-null   object        
 4   Subtitulo     1386 non-null   object        
 5   Class_etaria  1386 non-null   object        
 6   Telefone_pub  1386 non-null   object        
 7   Trad_libras   1386 non-null   object        
 8   Desc_sonora   1386 non-null   object        
 9   Obs           1386 non-null   object        
 10  Sites         1386 non-null   object        
 11  Id_evento     1386 non-null   float64       
 12  Frequencia    1386 non-null   object        
 13  Id_museu      1386 non-null   float64       
 14  Hora_inicio   1386 non-null   object        
 15  Hora_fim      1386 non-null   object  

Dropando a coluna 'Id_evento', já que é igual à coluna 'Id'

In [165]:
eventos[['Id','Id_evento']].head(5)

,Id,Id_evento
0,1325,1325.0
1,1327,1327.0
2,1328,1328.0
3,1329,1329.0
4,1330,1330.0


In [166]:
eventos.drop(columns={'Id_evento'}, inplace=True)

Convertendo o tipo de dados da coluna 'Id_museu' para int

In [167]:
#eventos.Id_museu.value_counts()
eventos.Id_museu = eventos.Id_museu.astype(int)

In [168]:
type(eventos.Id_museu[14])

numpy.int32

Posicionando as colunas

In [169]:
eventos = eventos[['Id','Id_museu','Nome_evento','Curta_desc','Longa_desc','Subtitulo','Preco','Inicio','Fim','Hora_inicio','Hora_fim','Descript','Class_etaria','Frequencia','Telefone_pub','Sites','Trad_libras','Desc_sonora','Obs']]
eventos.head(5)

,Id,Id_museu,Nome_evento,Curta_desc,Longa_desc,Subtitulo,Preco,Inicio,Fim,Hora_inicio,Hora_fim,Descript,Class_etaria,Frequencia,Telefone_pub,Sites,Trad_libras,Desc_sonora,Obs
0,1325,202940,Mostra | Diálogo Design: Polônia Brasil,"Com a curadoria de Magda Kochanowska, Gabriel ...","Dois países, duas culturas e duas histórias se...",Não informado,De terça a sexta: R$ 10 e R$ 5 (meia-entrada)....,2017-09-06,2017-10-22,10:00,18:00,Diariamente de 6 de setembro a 22 de outubro d...,Livre,Diário,(11) 3032-3727,http://www.mcb.org.br,Não informado,Não informado,Não informado
1,1327,202940,Educativo MCB | A casa estampada,"A artista Monica Schoenacker oferecerá, ao púb...","A artista Monica Schoenacker oferecerá, ao púb...",Não informado,Gratuito,2017-09-05,2017-09-19,11:00,13:00,"5 e 19 de setembro, terças-feiras às 11h",Livre,Semanal,(11) 3032-3727,http://www.mcb.org.br,Não informado,Não informado,Não informado
2,1328,202940,Educativo MCB | Desenho sua casa,"Marcos Gorgatti, artista educador do MCB, faz ...","Marcos Gorgatti, artista educador do MCB, faz ...",Não informado,Gratuito,2017-09-06,2017-09-20,10:30,12:00,"6 e 20 de setembro, quartas-feiras às 10h30",Livre,Semanal,(11) 3032-3727,http://www.mcb.org.br,Não informado,Não informado,Não informado
3,1329,202940,Educativo MCB | As palavras e as casas,A partir de relatos afetivos relacionados ao m...,A partir de relatos afetivos relacionados ao m...,Não informado,Gratuito,2017-09-13,2017-09-27,10:30,12:00,"13 e 27 de setembro, quartas-feiras às 10h30",Livre,Semanal,(11) 3032-3727,http://www.mcb.org.br,Não informado,Não informado,Não informado
4,1330,202940,Educativo MCB | Casa Bordada,"A partir de imagens de peças do acervo do MCB,...","A partir de imagens de peças do acervo do MCB,...",Não informado,Gratuito,2017-09-14,2017-09-14,13:30,15:00,Dia 14 de setembro de 2017 às 13:30,Livre,Uma vez,(11) 3032-3727,http://www.mcb.org.br,Não informado,Não informado,Não informado


### Criando o arquivo "eventos.parquet" 

In [170]:
eventos.to_parquet('eventos.parquet', compression='snappy', engine='fastparquet')

### Enviando o arquivo "museus.parquet" para o Blob

In [171]:
blob = BlobClient.from_connection_string(conn_str=chave, container_name=container, blob_name="eventos.parquet")
with open("eventos.parquet","rb") as data:
    blob.upload_blob(data, overwrite=True)
